In [1]:
import numpy as np
import matplotlib.pyplot as plt
import log_bessel as lb

# TESTS

In [2]:
v_to_test = np.array([0,3.15e-7, 2.62e-6, 1.3e-5, 9.2e-5, 0.0026,
                      0.0843, 0.17345, 1, 1.63, 7.42, 42.42424, 86.5,
                      113.8, 148.7565, 180.6, 246.3, 300.5, 513.6,
                      712.456, 714.456, 1235.6,8656, 15330.75,
                      37634.2 ,85323])

z_to_test = np.array([1.48e-7, 3.6e-6, 7.248e-5, 4.32e-4, 8.7e-3, 0.04523, 0.17532,
                      1, 3, 11.32465, 105.6, 1038.4, 4236, 11457.6, 62384, 105321.6,
                      158742.3, 196754,  1.98e6])

In [3]:
def logdiffexp(x1,x2):
    
    return x1+np.log1p(-np.exp(-(x1-x2)))

def forward_recursion(v,z):
    
    first = lb.log_bessel_k(v+2,z)[0]
    second = np.log(2) + np.log(v+1) - np.log(z) + lb.log_bessel_k(v+1,z)[0]
    
    return logdiffexp(first,second)

def backward_recursion(v, z):
    
    first = lb.log_bessel_k(v-2, z)[0]
    second = np.log(2)+np.log(v-1)-np.log(z)+lb.log_bessel_k(v-1, z)[0]
    
    return np.logaddexp(first, second)

def recursion_test(v, z):
    
    rhs = np.zeros(np.shape(v))
    
    i_forward = v <= 1
    i_backward = v > 1
    
    rhs[i_forward] = forward_recursion(v[i_forward], z)
    rhs[i_backward] = backward_recursion(v[i_backward], z)
    
    lhs = lb.log_bessel_k(v, z)[0]
    
    return lhs/rhs

In [4]:
v_0 = 1.63
z_0 = z_to_test[-2]

In [5]:
recursion_test(v_to_test, z_to_test[0])

array([       inf, 5.28243268, 4.42889071, 3.84527355, 3.1314961 ,
       1.90030317, 1.12232279, 1.06473451, 0.94282679, 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        ])

In [23]:
def rothwell_log_z_boundary(v):
    
    rothwell_max_log_z_over_v = 300

    return rothwell_max_log_z_over_v / (v-0.5)-np.log(2)

def method_indices(v, z):

    rothwell_max_v = 50
    rothwell_max_z = 100000
    rothwell_max_log_z_over_v = 300

    rothwell_1 = v < rothwell_max_v
    rothwell_2 = np.log(z) < rothwell_log_z_boundary(v)

    i_rothwell = np.logical_and(rothwell_1, rothwell_2)

    i_asymp_v = np.logical_and(np.sqrt(v+1) > z, ~i_rothwell)
    
    i_asymp_z = np.logical_and(~i_rothwell,
                               ~i_asymp_v)
    

    return i_rothwell, i_asymp_z, i_asymp_v

In [24]:
method_indices(v_to_test[0], z_to_test[0])

(False, False, True)

In [20]:
lb.method_indices(v_to_test[0], z_to_test[0])

(False, True, False)

In [22]:
v_to_test[0]
z_to_test[0]

1.48e-07